Purpose:
* Create a single mapping of criminal charges and characteristics, including:
    * Charge: _A text-based description of the crime. This is recorded different in different districts._ 
    * Massachusetts General Law (Where in MGL is this crime codified)
        * Chapter
        * Section
        * Paragraph (if applicable)
    * Misdemeanor / Felony
    * Sex offense per 100J
    * Murder offense
    * Additional criteria required to determine expungement eligibility
    
The additional criteria may be incomplete and subject to revisions, as it does not exist in a 'coded' state and will likely need to be extracted using regex.

This file will start with the charges from the Master Crime List, and will be built out over time to include alternative or additional charges from each prosecution dataset from each district. 

This process may need to be re-run as new datasets reveal new charges, new text descriptions of charges, new charge-extra criteria combinations, etc.

In [1]:
import pandas as pd
import regex as re

### Import data from Master List
This is the data from the `Added FBI Cat. and Expunge` tab of the **[Master Crime List offense with Expunge categories](https://docs.google.com/spreadsheets/d/11iD3ilejUW28NE6DdUaUkkp3PoPauhCj/edit#gid=579055210)** spreadsheet provided by CfJJ.

In [2]:
MCL = pd.read_csv('../data/raw/ExpungeCategories.csv')
# rename columns
MCL = MCL.rename(columns={"Now Legal?\n": "Now Legal", "Expungeable?": "Expungeable", 
                    "If no, why not?" :"Why not eligible", "Untruncated Offense" : "MCL_Offense"})
MCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Code                 1037 non-null   int64  
 1   MCL_Offense          1037 non-null   object 
 2   FBI Category         1037 non-null   object 
 3   MOST_SER_RANK        1025 non-null   float64
 4   OFFENSE_TYPE         1037 non-null   object 
 5   GRID                 1037 non-null   object 
 6   Penalty Type         1037 non-null   object 
 7   Mandatory Gun        15 non-null     object 
 8   Police Notification  35 non-null     object 
 9   SORB                 53 non-null     object 
 10  SDP                  53 non-null     object 
 11  Now Legal            0 non-null      float64
 12  Expungeable          1037 non-null   object 
 13  Why not eligible     318 non-null    object 
dtypes: float64(2), int64(1), object(11)
memory usage: 113.5+ KB


In [3]:
# Standardize values for 'no'
MCL['Expungeable'] = MCL['Expungeable'].str.strip().replace('NO', 'No')

In [4]:
# check that values for other columns are fairly consistent
for column in MCL.columns[7:]:
    print(MCL[column].value_counts(), '\n')


Yes                                14
Only if YO w/ an adult sentence     1
Name: Mandatory Gun, dtype: int64 

Yes    35
Name: Police Notification, dtype: int64 

Yes    53
Name: SORB, dtype: int64 

Yes    53
Name: SDP, dtype: int64 

Series([], Name: Now Legal, dtype: int64) 

Yes    719
No     318
Name: Expungeable, dtype: int64 

Ch 265 Felony                      151
178C Sex Offense                    30
Death or serious injury             20
Armed w dangerous weapon            18
Death or serious bodily injury      17
Ch 90 S 24 Violation                16
Ch 269 S10 a-d violation            15
S 121-131Q of Ch 140                14
Elderly/Disabled                     8
Armed w a dangerous weapon           6
Ch 6 S 178C Sex Offense              4
Ch 269 S 10E violation               3
Ch 123A S 1 Offense                  3
Order pursuant Ch 209A               3
Elderly                              3
Order Pursuan Ch 258E                1
Order pursuant Ch 258E               1


In [5]:
# view some offenses. next we are going to splice these into descriptin, chapter, section, and paragraph.

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 500
MCL[['MCL_Offense']][:100]

,MCL_Offense
0,A&b
1,A&B Ch. 265 S 13A(a)
2,A&b on a corrections officer
3,A&b on a public servant
4,A&B on child to coerce criminal conspiracy Ch. 265 S 44
5,"A&B on child to coerce criminal conspiracy, subsq. off. Ch. 265 S 44"
6,"A&B on child under 14, bodily injury Ch. 265 S 13J"
7,"A&B on child under 14, substantial bodily injury Ch. 265 S 13J"
8,A&b on child with injury
9,A&b on child with substantial injury


In [6]:
# Use regex to create new columns for Charge Description, Chapter, and Section
MCL['Description'] = None
MCL['Chapter'] = None
MCL['Section'] = None

for i in range(len(MCL)):
    try:
        MCL.loc[i, 'Description'] = re.search('.+?(?=\sCh.\s)', MCL.iloc[i]['MCL_Offense'])[0].upper()
    except:
        MCL.loc[i, 'Description'] = MCL.iloc[i]['MCL_Offense'].upper()
        
    try:
        MCL.loc[i, 'Chapter'] = re.search('(?<=Ch.\s)\d.*?(?=\sS)', MCL.iloc[i]['MCL_Offense'])[0]
    except:
        MCL.loc[i, 'Chapter'] = None
        
    try:
        MCL.loc[i, 'Section'] = re.search('(?<=\sS\s)(\d.*)', MCL.iloc[i]['MCL_Offense'])[0]
    except:
        MCL.loc[i, 'Section'] = None
        
MCL['Paragraph'] = None
section_paragraph = MCL['Section'].str.split("(", n = 1, expand = True) 
MCL['Section'] = section_paragraph[0]
MCL['Paragraph'] = section_paragraph[1]
MCL['Paragraph'] = '(' + MCL['Paragraph']

In [15]:

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
MCL[['Description', 'Chapter', 'Section', 'Paragraph']][:100]


,Description,Chapter,Section,Paragraph
0,A&B,None,None,NaN
1,A&B,265,13A,(a)
2,A&B ON A CORRECTIONS OFFICER,None,None,NaN
3,A&B ON A PUBLIC SERVANT,None,None,NaN
4,A&B ON CHILD TO COERCE CRIMINAL CONSPIRACY,265,44,NaN
5,"A&B ON CHILD TO COERCE CRIMINAL CONSPIRACY, SUBSQ. OFF.",265,44,NaN
6,"A&B ON CHILD UNDER 14, BODILY INJURY",265,13J,NaN
7,"A&B ON CHILD UNDER 14, SUBSTANTIAL BODILY INJURY",265,13J,NaN
8,A&B ON CHILD WITH INJURY,None,None,NaN
9,A&B ON CHILD WITH SUBSTANTIAL INJURY,None,None,NaN


In [19]:
MCL['Paragraph'] = MCL['Paragraph'].fillna('x')

In [23]:
x = MCL.groupby(['Chapter', 'Section', 'Paragraph', 'Expungeable']).size().unstack(fill_value=0)
x

Expungeable                           No  Yes
Chapter  Section      Paragraph              
119      1            x                0    3
         39           x                0    1
         63           x                0    1
12       11J          x                0    2
120      26           x                0    1
127      38B          x                0    1
131      43           x                0    1
         58           x                0    1
         66           x                0    1
         67           x                0    1
138      2            x                0    1
         22           x                0    1
         34           x                0    1
         34B          x                0    1
         34C          x                0    2
140      122B         x                1    0
         128          x                1    0
         129          x                2    0
         130          x                1    0
         131          (h)              0    1
         131C         x                1    0
         131E         x                1    0
         131H         x                1    0
         131I         x                1    0
         131J         x                1    0
         131L         (a)              2    0
         131M         x                2    0
140D     27           x                0    1
148      16           x                0    1
         35           x                0    1
         39           x                0    1
159      104          x                0    1
159A     16           x                0    1
         31           x                0    1
160      220          x                0    1
162      18           x                0    1
166      42A          x                0    1
169A     2-13         x                0    1
18       5B           x                0    1
208      34C          x                1    0
209A     3B           x                1    0
         7            x                2    0
22E      11           x                0    1
233      20H          x                0    1
         5            x                0    1
258E     9            x                1    0
264      11           x                0    1
         2            x                0    1
         3            x                0    1
265      1            x                1    0
         10           x                1    0
         11           x                1    0
         13           x                3    0
         13A          (a)              0    2
                      (b)              1    0
                      (b)(i)           1    0
                      (b)(ii)          1    0
                      (b)(iii)         1    0
         13B          x                1    0
         13B 3/4      x                1    0
         13B cent 1/2 x                1    0
         13C          x                2    0
         13D          x                0    1
         13D cent     x                1    0
         13F          x                4    0
         13G          x                1    0
         13H          x                2    0
         13I          x                0    1
         13J          x                2    1
         13K          (a cent)         1    0
                      (b)              1    0
                      (c)              1    0
                      (d cent)         1    0
         13L          x                0    1
         13M          x                4    0
         14           x                1    0
         15           x                1    0
         15A          (a)              6    0
                      (b)              3    0
                      (c)(i)           1    0
                      (c)(ii)          1    0
                      (c)(iii)         1    0
                      (c)(iv)          1    0
         15B          (a)              2    0
                      (b)              1    

In [24]:
x.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 401 entries, ('119', '1', 'x') to ('94C', '40', 'x')
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   No      401 non-null    int64
 1   Yes     401 non-null    int64
dtypes: int64(2)
memory usage: 10.3+ KB


In [22]:
MCL.groupby(['Chapter', 'Section', 'Paragraph', 'Penalty Type']).size().unstack(fill_value=0)

Penalty Type                          CIV  Felony  Misdemeanor
Chapter  Section      Paragraph                               
119      1            x                 0       0            3
         39           x                 0       0            1
         63           x                 0       0            1
12       11J          x                 0       1            1
120      26           x                 0       0            1
127      38B          x                 0       1            0
131      43           x                 0       0            1
         58           x                 0       0            1
         66           x                 0       0            1
         67           x                 0       0            1
138      2            x                 0       0            1
         22           x                 0       0            1
         34           x                 0       0            1
         34B          x                 0       0            1
         34C          x                 0       0            2
140      122B         x                 0       0            1
         128          x                 0       1            0
         129          x                 0       1            1
         130          x                 0       1            0
         131          (h)               0       1            0
         131C         x                 0       0            1
         131E         x                 0       1            0
         131H         x                 0       0            1
         131I         x                 0       1            0
         131J         x                 0       0            1
         131L         (a)               0       1            1
         131M         x                 0       2            0
140D     27           x                 0       0            1
148      16           x                 0       0            1
         35           x                 0       0            1
         39           x                 0       0            1
159      104          x                 0       0            1
159A     16           x                 1       0            0
         31           x                 0       1            0
160      220          x                 0       0            1
162      18           x                 0       0            1
166      42A          x                 0       1            0
169A     2-13         x                 0       0            1
18       5B           x                 0       0            1
208      34C          x                 0       0            1
209A     3B           x                 0       0            1
         7            x                 0       0            2
22E      11           x                 0       0            1
233      20H          x                 0       0            1
         5            x                 0       0            1
258E     9            x                 0       0            1
264      11           x                 0       1            0
         2            x                 0       1            0
         3            x                 0       1            0
265      1            x                 0       1            0
         10           x                 0       1            0
         11           x                 0       1            0
         13           x                 0       3            0
         13A          (a)               0       0            2
                      (b)               0       1            0
                      (b)(i)            0       1            0
                      (b)(ii)           0       1            0
                      (b)(iii)          0       1            0
         13B          x                 0       1            0
         13B 3/4      x                 0       1            0
         13B cent 1/2 x                 0       1            0
         13C          x        

In [ ]:
MCL.to_csv('MCLtest.csv', index=False)

PCD = pd.read_csv('../data/processed/prosecution_charges_detailed.csv', encoding='cp1252') 
PCD.rename(columns={"Expungeable.":"Expungeable"}, inplace=True)
columns = ['Charge', 'Chapter', 'Section', 'Expungeable']
PCD = PCD[columns]

df = MCL.merge(PCD, on=['Chapter', 'Section'], how='left', indicator=True)
df._merge.value_counts()

df = df[['MCL_Offense', 'Chapter', 'Section', 'Paragraph', 'Expungeable_x', 'Charge',  'Expungeable_y']]
df = df.drop_duplicates()

df.to_csv('mergetest.csv', index=False)

The merge by chapter and section is very coarse. Dawn didn't keep the original charge from the MCL. Took the MCL, dropped the text description -- stripped down to chapter, section, expungeable. Dropped duplicates. While many chapter+sections have only expungeable or only nonexpungeable offenses, there are chapters+sections that encompass both expungeable and nonexpungeable offenses.
See x = MCL.groupby(['Chapter', 'Section', 'Paragraph', 'Expungeable']).size().unstack(fill_value=0)

Same for misdemeanor/felony.

We identified problems where the same prosecution_ data row linked to both "yes" and "no" for expungeable. 

Who's to say whether the matches that have only one expungeable value per chapter+section were actually correct? 

## next steps

validate current mapping:

1. get mapping of unique chapter/section/paragraph/expungeable
1. get mapping of unique chapter/section/paragraph/misdemeanor_felony
1. counts of most common chapter/section -- verify top % manually

try new methods:

1. see how the new complaint file stands up -- what is the match rate of this to the prosecution data, matching by string?
1. try to build a crosswalk of chapters and sections that are not expungeable, building directly from the 100J statutes
1. try chapter/section/paragarph/extra_criteria

or, stick with the mapping created in MA_Data-2_MergeCharges_alt, and just build from there

1. merge in misdemeanor/felony, merging on the text description